# Basis Recognition (Symbol-Supervised Model)

### Imports

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, minmax_scale
import torch.nn as nn
from Data_reading import*
import torch.optim as optim
import torch.nn.functional as F

### Data Dawnload and Scaling

In [ ]:
x = Symbol('x')

support = support_reading()
X_lin, Y_complexity_lin, GT_function_lin ,labels_one_hot_lin = read_data(
    support, dir = r"C:\Users\lbg\OneDrive - CSEM S.A\Bureau\Pytorch\NEW_EQ_LEARN\Data\Binomials")

X_lin = minmax_scale(X_lin,axis = 1)

X_lin_train, labels_one_hot_lin_train, GT_function_lin_train = X_lin[:8000], labels_one_hot_lin[:8000,:-1], GT_function_lin[:8000]
X_lin_test, labels_one_hot_lin_test, GT_function_lin_test = X_lin[8000:], labels_one_hot_lin[8000:,:-1], GT_function_lin[8000:]

labels_one_hot_lin_train.shape

In [ ]:
X_lin_train = np.expand_dims(X_lin_train,1)
X_lin_test = np.expand_dims(X_lin_test,1)
print(X_lin_test.shape)
print(X_lin_train.shape)

### Dataset Iterator Class

In [ ]:
class dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, dataset,  labels, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataset = dataset
        self.transform = transform
        self.label = labels

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        data = self.dataset[idx, :].astype('float32')
        label1 = self.label[idx, :]
        data = torch.from_numpy(data)
        label1 = torch.from_numpy(label1.astype('float32'))

        if self.transform:
            data = self.transform(data)

        return data,label1


### Train-Validation Split

In [ ]:
dataset_train = dataset(X_lin_train, labels_one_hot_lin_train)
dataset_test_h = dataset(X_lin_test, labels_one_hot_lin_test)
batch_size = 100


valid_size = 0.20
num_train = len(dataset_train)
num_test_h = len(dataset_test_h)
indices = list(range(num_train))
test_idx_h = list(range(num_test_h))
np.random.shuffle(test_idx_h)
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler_h = SubsetRandomSampler(test_idx_h)


train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
    sampler=train_sampler, num_workers=0)
valid_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=0)
test_loader_h = torch.utils.data.DataLoader(dataset_test_h, batch_size=batch_size, 
    sampler=test_sampler_h, num_workers=0)

### Network Model

In [ ]:
# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 9, 6, padding=1)
        self.bn1 = nn.BatchNorm1d(num_features=3)
        self.conv2 = nn.Conv1d(9, 9, 6, padding=1)
        self.bn2 = nn.BatchNorm1d(num_features=3)
        self.conv3 = nn.Conv1d(9, 9, 6, padding=1)
        self.bn3 = nn.BatchNorm1d(num_features=3)
        self.conv4 = nn.Conv1d(9, 9, 6, padding=1)
        self.bn4 = nn.BatchNorm1d(num_features=3)
        self.fc1 = nn.Linear(3*2958, 500)
        self.fc2 = nn.Linear(500, 7)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.4)
        self.dropout3 = nn.Dropout(0.4)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.4)

    def forward(self, x):
        x = F.relu((self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu((self.conv2(x)))
        x = self.dropout2(x)
        x = F.relu((self.conv3(x)))
        x = self.dropout3(x)
        x = F.relu((self.conv4(x)))
        x = self.dropout4(x)
        x = x.view(-1, x.shape[2]*9)
        x = F.relu(self.fc1(x))
        x = self.dropout5(x)
        x = F.relu(self.fc2(x))
        return x




### Training

In [ ]:
# create a complete NN
model = Net()
model.float()
print(model)

criterion = nn.MSELoss(reduction='mean')

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 0)

# number of epochs to train the model
n_epochs = 50



# track change in validation loss
valid_loss_min = np.Inf 


print('Do you want to retrain the model?')
resp = input()
if resp == 'yes':
    print('Do you want to save the model parameters?')
    save = input()
    for epoch in range(1, n_epochs+1):

        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train()
        for data, target in train_loader:
            # move tensors to GPU if CUDA is available
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            print(output)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)


        ######################    
        # validate the model #
        ######################
        model.eval()
        for data, target in valid_loader:
            # move tensors to GPU if CUDA is available
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            #_, pred = torch.max(output, 1)   
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)

        # calculate average losses
        train_loss = train_loss/len(train_loader.sampler)
        valid_loss = valid_loss/len(valid_loader.sampler)


        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss))
        
        if save == 'yes':
            # save model if validation loss has decreased
            if valid_loss <= valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
                torch.save(model.state_dict(), r"C:\Users\lbg\OneDrive - CSEM S.A\Bureau\Pytorch\NEW_EQ_LEARN\Saved_Models\CNN1.pt")
                valid_loss_min = valid_loss
else:
    print('Model not trained')
    pass

### Load Model and Evaluate it

In [ ]:
model.eval()
model.load_state_dict(torch.load(r"C:\Users\lbg\OneDrive - CSEM S.A\Bureau\Pytorch\NEW_EQ_LEARN\Saved_Models\CNN1.pt"))
Y_hat_test = model(torch.from_numpy(X_lin_test.astype('float32')))
Y_hat_val = model(torch.from_numpy(X_lin_train[valid_idx].astype('float32')))
Y_hat_tr = model(torch.from_numpy(X_lin_train[train_idx].astype('float32')))
Y_hat_train = model(torch.from_numpy(X_lin_train.astype('float32')))

In [ ]:
x = Symbol('x')
basis_functions = np.array([sin(x),exp(x), x, exp(-x), 1/x, sin(x**2), exp(-x**2),1])
yb = torch.Tensor([[lambdify(x,basis_functions[j])(i) for i in support] for j in range(len(basis_functions)-1)])


In [ ]:
ys = torch.transpose(torch.Tensor(X_lin_train),1,2)[:100]
x = torch.eye(7)
x = x.reshape((1, 7, 7))
y = x.repeat(100, 1, 1)
a = torch.nn.Threshold(0.5,0)
yy = a(Y_hat_test[:100])
s = torch.unsqueeze(yy,1)
q = s*y
c = torch.bmm(torch.unsqueeze(yb.T,0).repeat(100,1,1),q)
d = torch.bmm(torch.transpose(c,1,2),c) + 0.00001*torch.eye(7)
e = torch.inverse(d)
f = torch.bmm(e,torch.transpose(c,1,2))
print(torch.bmm(f,ys))

### Look at  perfromances

In [ ]:
#15,150,120,190,51,173
%matplotlib notebook

temp = Y_hat_test>0.5
Y_hat_test = temp.numpy().astype(int)
temp = Y_hat_train>0.5
Y_hat_train = temp.numpy().astype(int)

print('Overall Accuracy')
print(1-sum([False in (labels_one_hot_lin_test[i] == Y_hat_test[i]) for i in range(len(Y_hat_test))])/len(Y_hat_test))
print(1-sum([False in (labels_one_hot_lin_train[i] == Y_hat_train[i]) for i in range(len(Y_hat_train))])/len(Y_hat_train))
print()

basis_functions = np.array([sin(x),exp(x), x, exp(-x), 1/x, sin(x**2), exp(-x**2),1])
sample = 211
print('True Equation:')
print(GT_function_lin_test[sample])
print()
print('Prediction')
print(Y_hat_test[sample])
print()
print('Ground Truth')
print(labels_one_hot_lin_test[sample])
y = [lambdify(x,GT_function_lin_test[sample])(i) for i in support]
plt.plot(support,y)

In [ ]:
%matplotlib notebook
plt.plot(support,X_lin_test[493,0])

### Single-basis Performance

In [ ]:
print('Test Set')
for i in range(7):
    print(sum(labels_one_hot_lin_test[:,i] == Y_hat_test[:,i])/len(Y_hat_test))

print()    
    

print('Training Set')    
for i in range(7):
    print(sum(labels_one_hot_lin_train[:,i] == Y_hat_train[:,i])/len(Y_hat_train))

### Confusion Matrix

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
sums = [sum(labels_one_hot_lin_test[:,i]) for i in range(8)]
confusion = multilabel_confusion_matrix(labels_one_hot_lin_test, Y_hat_test)
for i in range(8):
    print(confusion[i]/sums[i])
    print()
